# Generative OpenAIモジュールを使用したWeaviateでの生成検索

このノートブックは以下のシナリオ向けに準備されています：
* データが既にWeaviateに格納されている
* Generative OpenAIモジュール（[generative-openai](https://weaviate.io/developers/weaviate/modules/reader-generator-modules/generative-openai)）を使用してWeaviateを利用したい

## 前提条件

このクックブックは生成検索の例のみを扱っており、設定やデータインポートについては説明していません。

このクックブックを最大限に活用するために、まず[Getting Startedクックブック](./getting-started-with-weaviate-and-openai.ipynb)を完了してください。そこでWeaviateの操作の基本を学び、デモデータをインポートします。

チェックリスト：
* [Getting Startedクックブック](./getting-started-with-weaviate-and-openai.ipynb)を完了している
* `Weaviate`インスタンスを作成している
* `Weaviate`インスタンスにデータをインポートしている
* [OpenAI APIキー](https://beta.openai.com/account/api-keys)を持っている

## OpenAI API キーの準備

`OpenAI API key`は、インポート時のデータのベクトル化とクエリの実行に使用されます。

OpenAI API キーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、環境変数として`OPENAI_API_KEY`に追加してください。

In [ ]:
# Export OpenAI API Key
!export OPENAI_API_KEY="your key"

In [ ]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'your-key-goes-here'

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

## Weaviateインスタンスに接続する

このセクションでは、以下を行います：

1. 環境変数`OPENAI_API_KEY`をテストする – [#Prepare-your-OpenAI-API-key](#Prepare-your-OpenAI-API-key)のステップを完了していることを**確認してください**
2. `OpenAI API Key`を使用してWeaviateに接続する
3. クライアント接続をテストする

### クライアント

このステップの後、`client`オブジェクトはすべてのWeaviate関連の操作を実行するために使用されます。

In [ ]:
import weaviate
from datasets import load_dataset
import os

# Connect to your Weaviate instance
client = weaviate.Client(
    url="https://your-wcs-instance-name.weaviate.network/",
    # url="http://localhost:8080/",
    auth_client_secret=weaviate.auth.AuthApiKey(api_key="<YOUR-WEAVIATE-API-KEY>"), # comment out this line if you are not using authentication for your Weaviate instance (i.e. for locally deployed instances)
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")
    }
)

# Check if your instance is live and ready
# This should return `True`
client.is_ready()

## Generative Search
Weaviateは[Generative Search OpenAI](https://weaviate.io/developers/weaviate/modules/reader-generator-modules/generative-openai)モジュールを提供しており、Weaviateインスタンスに保存されたデータに基づいて応答を生成します。

Generative Searchクエリの構築方法は、Weaviateの標準的なセマンティック検索クエリと非常に似ています。

例えば：
* "Articles"で検索
* "title"、"content"、"url"を返す
* "football clubs"に関連するオブジェクトを探す
* 結果を5つのオブジェクトに制限

```
    result = (
        client.query
        .get("Articles", ["title", "content", "url"])
        .with_near_text("concepts": "football clubs")
        .with_limit(5)
        # generative query will go here
        .do()
    )
```

次に、`with_generate()`関数を追加してGenerative変換を適用できます。`with_generate`は以下のいずれかを受け取ります：
- `single_prompt` - 返された各オブジェクトに対して応答を生成
- `grouped_task` - 返されたすべてのオブジェクトから単一の応答を生成

In [ ]:
def generative_search_per_item(query, collection_name):
    prompt = "Summarize in a short tweet the following content: {content}"

    result = (
        client.query
        .get(collection_name, ["title", "content", "url"])
        .with_near_text({ "concepts": [query], "distance": 0.7 })
        .with_limit(5)
        .with_generate(single_prompt=prompt)
        .do()
    )
    
    # Check for errors
    if ("errors" in result):
        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")
        raise Exception(result["errors"][0]['message'])
    
    return result["data"]["Get"][collection_name]

In [ ]:
query_result = generative_search_per_item("football clubs", "Article")

for i, article in enumerate(query_result):
    print(f"{i+1}. { article['title']}")
    print(article['_additional']['generate']['singleResult']) # print generated response
    print("-----------------------")

In [79]:
def generative_search_group(query, collection_name):
    generateTask = "Explain what these have in common"

    result = (
        client.query
        .get(collection_name, ["title", "content", "url"])
        .with_near_text({ "concepts": [query], "distance": 0.7 })
        .with_generate(grouped_task=generateTask)
        .with_limit(5)
        .do()
    )
    
    # Check for errors
    if ("errors" in result):
        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")
        raise Exception(result["errors"][0]['message'])
    
    return result["data"]["Get"][collection_name]

In [ ]:
query_result = generative_search_group("football clubs", "Article")

print (query_result[0]['_additional']['generate']['groupedResult'])

お疲れ様でした。これで独自のベクトルデータベースを構築し、埋め込みを使用して様々な素晴らしいことができるようになりました - お楽しみください！より複雑なユースケースについては、このリポジトリの他のクックブック例を引き続き参照してください。